In [40]:
import pandas as pd
import geopandas as gpd
import h3
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

On a déjà les données des Landes uniquement, en préselectionnant les indices que l'on souhaite regarder.

In [2]:
path_RRT_Vent_Meteo_Landes = '/Users/camillemodeste/Documents/Workflow/Pyronear/hackathon-meteo-france/data/processed/RRT_Vent_Meteo_Landes.geojson'
path_SIM_Meteo_Landes = '/Users/camillemodeste/Documents/Workflow/Pyronear/hackathon-meteo-france/data/processed/SIM_Meteo_Landes.geojson'
path_CLC_simplified_Landes = '/Users/camillemodeste/Documents/Workflow/Pyronear/hackathon-meteo-france/data/processed/CLC_simplified_Landes.geojson'

### Import des données et focus sur les années 2020 à 2022

In [3]:
gdf_RRT_Vent_Meteo_Landes = gpd.read_file(path_RRT_Vent_Meteo_Landes)  
gdf_SIM_Meteo_Landes = gpd.read_file(path_SIM_Meteo_Landes)  
gdf_CLC_simplified_Landes = gpd.read_file(path_CLC_simplified_Landes)  

In [43]:
gdf_CLC_simplified_Landes

,ID,CODE_12,AREA_HA,geometry
0,FR-108973,Zone agricole,37.556156,"POLYGON ((-1.15849 43.69093, -1.15787 43.69083..."
1,FR-31523,Zone industrielle,36.549172,"POLYGON ((-1.12492 43.68834, -1.12702 43.68766..."
2,FR-228765,Forêt,93.393277,"POLYGON ((-1.32439 43.68382, -1.32516 43.68262..."
3,FR-37103,Zone agricole,118.046729,"POLYGON ((-1.32596 43.67639, -1.32623 43.67600..."
4,FR-151303,Zone agricole,47.595193,"POLYGON ((-1.33086 43.68280, -1.33086 43.68277..."
...,...,...,...,...
52005,FR-161114,Zone agricole,39.099535,"POLYGON ((0.65255 46.41261, 0.65474 46.41016, ..."
52006,FR-190951,Forêt,306.701103,"POLYGON ((0.64267 46.38848, 0.64469 46.38702, ..."
52007,FR-125707,Zone agricole,812.496416,"POLYGON ((0.67556 46.34454, 0.67545 46.34338, ..."
52008,FR-190516,Forêt,62.304810,"POLYGON ((0.64676 46.30419, 0.64748 46.30364, ..."


Focus sur les années 2020-2021-2022 :

In [4]:
gdf_RRT_Vent_Meteo_Landes = gdf_RRT_Vent_Meteo_Landes[gdf_RRT_Vent_Meteo_Landes.AAAAMMJJ >= 20200101]
gdf_SIM_Meteo_Landes = gdf_SIM_Meteo_Landes[gdf_SIM_Meteo_Landes.DATE >= 20200101]


In [41]:
gdf_SIM_Meteo_Landes

,DATE,PRENEI_Q,PRELIQ_Q,SWI_Q,TSUP_H_Q,longitude,latitude,geometry,h3
0,20200101,0.0,0.2,0.885,11.6,-1.373709,43.579189,POINT (-1.37371 43.57919),861849cefffffff
1,20200102,0.0,0.2,0.878,10.5,-1.373709,43.579189,POINT (-1.37371 43.57919),861849cefffffff
2,20200103,0.0,1.9,0.872,10.3,-1.373709,43.579189,POINT (-1.37371 43.57919),861849cefffffff
3,20200104,0.0,0.7,0.873,11.2,-1.373709,43.579189,POINT (-1.37371 43.57919),861849cefffffff
4,20200105,0.0,0.2,0.868,4.5,-1.373709,43.579189,POINT (-1.37371 43.57919),861849cefffffff
...,...,...,...,...,...,...,...,...,...
285943,20240225,0.0,34.4,1.111,11.6,-0.025280,44.480560,POINT (-0.02528 44.48056),863964d97ffffff
285944,20240226,0.0,13.7,1.180,10.5,-0.025280,44.480560,POINT (-0.02528 44.48056),863964d97ffffff
285945,20240227,0.0,0.0,1.181,8.0,-0.025280,44.480560,POINT (-0.02528 44.48056),863964d97ffffff
285946,20240228,0.0,0.2,1.152,11.1,-0.025280,44.480560,POINT (-0.02528 44.48056),863964d97ffffff


Uniformisation des noms des df :

In [33]:
gdf_RRT_Vent_Meteo_Landes = gdf_RRT_Vent_Meteo_Landes.rename(columns={'LAT': 'latitude', 'LON': 'longitude','AAAAMMJJ':'DATE'})
gdf_RRT_Vent_Meteo_Landes['FXY']=gdf_RRT_Vent_Meteo_Landes['FXY'].fillna(0)

285948

Dans quel polygone CLC se trouve le point SIM ?

In [ ]:
nb_lignes = gdf_SIM_Meteo_Landes.shape[0]

def associate_points_with_polygons(point_gdf, polygon_gdf):
    # Create a spatial index for faster spatial queries
    spatial_index = polygon_gdf.sindex
    c=1
    # Iterate through points and find the containing polygon
    for index, point in point_gdf.iterrows():
        print(100*c/nb_lignes,"% progress")
        c=c+1
        # Find the index of the polygon that contains the point
        possible_matches_index = list(spatial_index.intersection(point.geometry.bounds))
        possible_matches = polygon_gdf.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.contains(point.geometry)]
        
        # If a precise match is found, assign the polygon name to the point
        if not precise_matches.empty:
            point_gdf.at[index, 'CLC land classification'] = precise_matches.iloc[0]['CODE_12']
    
    return point_gdf

# Sample code to create some example data (Remove this block if you have your own GeoDataFrames)

# Associate points with polygons
gdf_SIM_Meteo_Landes['CLC land classification']=''
gdf_SIM_Meteo_Landes = associate_points_with_polygons(gdf_SIM_Meteo_Landes, gdf_CLC_simplified_Landes)

# Print the result
print(gdf_SIM_Meteo_Landes)

0.00034971393400198635 % progress
0.0006994278680039727 % progress
0.0010491418020059592 % progress
0.0013988557360079454 % progress
0.0017485696700099318 % progress
0.0020982836040119184 % progress
0.0024479975380139046 % progress
0.002797711472015891 % progress
0.0031474254060178774 % progress
0.0034971393400198636 % progress
0.0038468532740218503 % progress
0.004196567208023837 % progress
0.004546281142025823 % progress
0.004895995076027809 % progress
0.005245709010029796 % progress
0.005595422944031782 % progress
0.005945136878033768 % progress
0.006294850812035755 % progress
0.0066445647460377415 % progress
0.006994278680039727 % progress
0.007343992614041714 % progress
0.0076937065480437005 % progress
0.008043420482045687 % progress
0.008393134416047674 % progress
0.008742848350049659 % progress
0.009092562284051645 % progress
0.009442276218053632 % progress
0.009791990152055619 % progress
0.010141704086057605 % progress
0.010491418020059592 % progress
0.010841131954061578 % prog

### Merge géographique des données

In [34]:
resolution = 6 # résolution de 6km, racine de 7 puissance 9 (puissance 0 car 15-6=9)

def carroyage_h3(lat, lng):
    return h3.geo_to_h3(lat, lng, resolution)

gdf_RRT_Vent_Meteo_Landes['h3'] = gdf_RRT_Vent_Meteo_Landes.apply(lambda x : carroyage_h3(x['latitude'], x['longitude']), axis=1)
gdf_SIM_Meteo_Landes['h3'] = gdf_SIM_Meteo_Landes.apply(lambda x : carroyage_h3(x['latitude'], x['longitude']), axis=1)

In [35]:
gdf_Meteo_Landes = pd.merge(gdf_SIM_Meteo_Landes, gdf_RRT_Vent_Meteo_Landes,how="outer", on=["h3","DATE"])

In [36]:
gdf_Meteo_Landes = gdf_Meteo_Landes.drop(columns=['longitude_x','latitude_x','geometry_x','latitude_y','longitude_y','geometry_y'])
gdf_Meteo_Landes["polygon"] = gdf_Meteo_Landes.apply(lambda x: h3.h3_to_geo_boundary(x['h3']), axis=1)
gdf_Meteo_Landes

,DATE,PRENEI_Q,PRELIQ_Q,SWI_Q,TSUP_H_Q,h3,FXY,polygon
0,20200101,0.0,0.2,0.885,11.6,861849cefffffff,NaN,"((43.597410221986316, -1.3429476411059258), (4..."
1,20200102,0.0,0.2,0.878,10.5,861849cefffffff,NaN,"((43.597410221986316, -1.3429476411059258), (4..."
2,20200103,0.0,1.9,0.872,10.3,861849cefffffff,NaN,"((43.597410221986316, -1.3429476411059258), (4..."
3,20200104,0.0,0.7,0.873,11.2,861849cefffffff,NaN,"((43.597410221986316, -1.3429476411059258), (4..."
4,20200105,0.0,0.2,0.868,4.5,861849cefffffff,NaN,"((43.597410221986316, -1.3429476411059258), (4..."
...,...,...,...,...,...,...,...,...
297091,20221227,NaN,NaN,NaN,NaN,861849c1fffffff,0.0,"((43.70959689639052, -1.3982353009154476), (43..."
297092,20221228,NaN,NaN,NaN,NaN,861849c1fffffff,0.0,"((43.70959689639052, -1.3982353009154476), (43..."
297093,20221229,NaN,NaN,NaN,NaN,861849c1fffffff,0.0,"((43.70959689639052, -1.3982353009154476), (43..."
297094,20221230,NaN,NaN,NaN,NaN,861849c1fffffff,0.0,"((43.70959689639052, -1.3982353009154476), (43..."
